In [77]:
import os
import re

import numpy as np
import scanpy as sc
import torch
import tifffile
import yaml
from torchio.transforms import Resize
from einops import rearrange

import napari

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [85]:
from mushroom.data.multiplex import extract_ome_tiff
from mushroom.data.visium import adata_from_visium, pixels_per_micron
from mushroom.visualization.utils import calculate_target_visualization_shape

In [8]:
config_filepath = '/Users/erikstorrs/Documents/ding/mushroom/data/examples/HT397B1_v1/mushroom_outputs/multiplex/config_local.yaml'
outputs_filepath = '/Users/erikstorrs/Documents/ding/mushroom/data/examples/HT397B1_v1/mushroom_outputs/multiplex/outputs.pt'


In [9]:
data = torch.load(outputs_filepath)
data.keys()

dict_keys(['recon_embs', 'recon_imgs', 'true_imgs', 'cluster_distances', 'cluster_distance_volume', 'cluster_centroids', 'cluster_ids'])

In [10]:
for k, v in data.items():
    print(k, v.shape)

recon_embs torch.Size([4, 1024, 60, 60])
recon_imgs torch.Size([4, 24, 1920, 1920])
true_imgs torch.Size([4, 24, 1920, 1920])
cluster_distances torch.Size([4, 60, 60, 20])
cluster_distance_volume torch.Size([30, 60, 60, 20])
cluster_centroids torch.Size([20, 1024])
cluster_ids torch.Size([4, 60, 60])


In [66]:
config = yaml.safe_load(open(config_filepath))
sections = config['sections']
sections

[{'data': [{'dtype': 'he',
    'filepath': '/Users/erikstorrs/Documents/ding/mushroom/data/examples/HT397B1_v1/registration/registered/s0_he.tif'},
   {'dtype': 'visium',
    'filepath': '/Users/erikstorrs/Documents/ding/mushroom/data/examples/HT397B1_v1/registration/registered/s0_visium.h5ad'}],
  'id': 's0',
  'position': 0},
 {'data': [{'dtype': 'multiplex',
    'filepath': '/Users/erikstorrs/Documents/ding/mushroom/data/examples/HT397B1_v1/registration/registered/s1_multiplex.ome.tiff'}],
  'id': 's1',
  'position': 1},
 {'data': [{'dtype': 'multiplex',
    'filepath': '/Users/erikstorrs/Documents/ding/mushroom/data/examples/HT397B1_v1/registration/registered/s2_multiplex.ome.tiff'}],
  'id': 's2',
  'position': 11},
 {'data': [{'dtype': 'he',
    'filepath': '/Users/erikstorrs/Documents/ding/mushroom/data/examples/HT397B1_v1/registration/registered/s3_he.tif'},
   {'dtype': 'visium',
    'filepath': '/Users/erikstorrs/Documents/ding/mushroom/data/examples/HT397B1_v1/registration/r

In [75]:
target_shape = calculate_target_visualization_shape(config, data, microns_per_section=5) # (x y z)
target_shape

(1920, 1920, 34)

In [78]:
distance_volume = rearrange(data['cluster_distance_volume'], 'z y x d -> d x y z')
transform = Resize(target_shape)
distance_volume = transform(distance_volume)
distance_volume.shape

torch.Size([20, 1920, 1920, 34])

In [ ]:
# what do we need
# downsample = .1 -> further downsample from initial downsample that mushroom does
# downsample_shape -> (x y z)
# the following data 
## methods to reconstruc

In [15]:
config = yaml.safe_load(open(config_filepath))
sections = config['sections']
sections

[{'data': [{'dtype': 'he',
    'filepath': '/Users/erikstorrs/Documents/ding/mushroom/data/examples/HT397B1_v1/registration/registered/s0_he.tif'},
   {'dtype': 'visium',
    'filepath': '/Users/erikstorrs/Documents/ding/mushroom/data/examples/HT397B1_v1/registration/registered/s0_visium.h5ad'}],
  'id': 's0',
  'position': 0},
 {'data': [{'dtype': 'multiplex',
    'filepath': '/Users/erikstorrs/Documents/ding/mushroom/data/examples/HT397B1_v1/registration/registered/s1_multiplex.ome.tiff'}],
  'id': 's1',
  'position': 1},
 {'data': [{'dtype': 'multiplex',
    'filepath': '/Users/erikstorrs/Documents/ding/mushroom/data/examples/HT397B1_v1/registration/registered/s2_multiplex.ome.tiff'}],
  'id': 's2',
  'position': 11},
 {'data': [{'dtype': 'he',
    'filepath': '/Users/erikstorrs/Documents/ding/mushroom/data/examples/HT397B1_v1/registration/registered/s3_he.tif'},
   {'dtype': 'visium',
    'filepath': '/Users/erikstorrs/Documents/ding/mushroom/data/examples/HT397B1_v1/registration/r

In [33]:
s = sections[0]
fp = [entry['filepath'] for entry in s['data'] if entry['dtype']=='visium'][0]
adata = sc.read_h5ad(fp)
adata

DEBUG:h5py._conv:Creating converter from 3 to 5


AnnData object with n_obs × n_vars = 3058 × 17943
    obs: 'in_tissue', 'array_row', 'array_col'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'

In [38]:
scalefactors = next(iter(adata.uns['spatial'].values()))['scalefactors']
pixels_per_micron = scalefactors['spot_diameter_fullres'] / 65. # each spot is 65 microns wide
pixels_per_micron

2.3223207975307805

In [44]:
scalef = scalefactors['tissue_hires_scalef']
hires_shape = next(iter(adata.uns['spatial'].values()))['images']['hires'].shape
fullres_shape = (int(hires_shape[0] / scalef), int(hires_shape[1] / scalef))
fullres_shape

(18643, 18381)

In [53]:
step_size = pixels_per_micron * 5 * config['learner_kwargs']['scale']
z_max = int(step_size * np.max([entry['position'] for entry in sections]))
z_max

34

In [54]:
target_shape = (data['true_imgs'].shape[-2], data['true_imgs'].shape[-1], z_max)
target_shape

(1920, 1920, 34)

In [45]:
config.keys()

dict_keys(['chkpt_filepath', 'cluster_kwargs', 'dtype', 'learner_kwargs', 'sae_kwargs', 'sections', 'train_kwargs'])

In [ ]:
step_size = 

In [42]:
scalefactors

{'fiducial_diameter_fullres': 243.84368374073196,
 'spot_diameter_fullres': 150.95085183950073,
 'tissue_hires_scalef': 0.10727887,
 'tissue_lowres_scalef': 0.032183662}

In [58]:
from ome_types import from_xml
tif = tifffile.TiffFile('/Users/erikstorrs/Documents/ding/mushroom/data/examples/HT397B1_v1/registration/registered/s2_multiplex.ome.tiff')
ome = from_xml(tif.ome_metadata)
im = ome.images[0]

In [61]:
im.pixels.physical_size_x

2.3223207975307805

In [63]:
sections[0]['data']

[{'dtype': 'he',
  'filepath': '/Users/erikstorrs/Documents/ding/mushroom/data/examples/HT397B1_v1/registration/registered/s0_he.tif'},
 {'dtype': 'visium',
  'filepath': '/Users/erikstorrs/Documents/ding/mushroom/data/examples/HT397B1_v1/registration/registered/s0_visium.h5ad'}]

In [79]:
adata

AnnData object with n_obs × n_vars = 3058 × 17943
    obs: 'in_tissue', 'array_row', 'array_col'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'

In [82]:
d = next(iter(adata.uns['spatial'].values()))
d.keys()

dict_keys(['images', 'metadata', 'scalefactors'])

In [83]:
d['scalefactors']

{'fiducial_diameter_fullres': 243.84368374073196,
 'spot_diameter_fullres': 150.95085183950073,
 'tissue_hires_scalef': 0.10727887,
 'tissue_lowres_scalef': 0.032183662}

In [84]:
d['images'].keys()

dict_keys(['hires', 'lowres'])

In [86]:
pixels_per_micron = pixels_per_micron(adata)
pixels_per_micron

2.3223207975307805

In [87]:
scale = .1
downsample = .1
downsample_ppm = pixels_per_micron * scale * downsample
downsample_ppm

0.023223207975307806

In [89]:
tif.ome_metadata

'<?xml version="1.0" encoding="UTF-8"?><OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd" UUID="urn:uuid:13a44b72-411d-11ee-ab10-ac1f6b9e3060"  Creator="tifffile.py 2023.8.12"><Image ID="Image:0" Name="Image0"><Pixels ID="Pixels:0" DimensionOrder="XYZCT" Type="uint8" SizeX="18370" SizeY="18640" SizeZ="1" SizeC="25" SizeT="1" PhysicalSizeX="2.3223207975307805" PhysicalSizeXUnit="µm" PhysicalSizeY="2.3223207975307805" PhysicalSizeYUnit="µm"><Channel ID="Channel:0:0" SamplesPerPixel="1" Name="DAPI"><LightPath/></Channel><Channel ID="Channel:0:1" SamplesPerPixel="1" Name="CD8"><LightPath/></Channel><Channel ID="Channel:0:2" SamplesPerPixel="1" Name="Her2 (D)"><LightPath/></Channel><Channel ID="Channel:0:3" SamplesPerPixel="1" Name="GATA3 (D)"><LightPath/></Channel><Channel ID="Channel:0:4" SamplesP

In [91]:
im.pixels.size_x, im.pixels.size_y

(18370, 18640)

In [92]:
tif = tifffile.TiffFile(
    '/Users/erikstorrs/Documents/ding/mushroom/data/examples/HT397B1_v1/registration/registered/s0_he.tif'
)

In [94]:
p = next(iter(tif.pages))
p

<tifffile.TiffPage 0 @8>

In [95]:
p.shape

(18643, 18378, 3)

In [96]:
scalefactors

{'fiducial_diameter_fullres': 243.84368374073196,
 'spot_diameter_fullres': 150.95085183950073,
 'tissue_hires_scalef': 0.10727887,
 'tissue_lowres_scalef': 0.032183662}